# Phase 1: LSTM motion predictor training

Train the improved LSTM predictor on the same datasets and setup as the transformer (phase1_improved). Uses `GTSequenceDataset` (src, trg, gt_src, gt_trg) and `LossFunction` with CIoU + confidence.

In [ ]:
from dataset import GTSequenceDataset
from torch.utils.data import DataLoader, ConcatDataset

SEQ_IN_LEN = 30
SEQ_OUT_LEN = 20
SEQ_TOTAL_LEN = 50
BATCH_SIZE = 512
STEPS = 4
NOISE_COEFFICIENT = 0.15
NOISE_PROB = 0

BASE_DIR = '../../Datasets/'
train_dataset = GTSequenceDataset.from_roots([
    # f'{BASE_DIR}/SportsMOT/train',
    # f'{BASE_DIR}DanceTrack/train',
    f'{BASE_DIR}MOT17/train',
    # f'{BASE_DIR}MOT20/train'
], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB)

val_dataset = GTSequenceDataset.from_roots([
    # f'{BASE_DIR}/SportsMOT/val',
    # f'{BASE_DIR}DanceTrack/val',
    f'{BASE_DIR}MOT17/val',
    # f'{BASE_DIR}MOT20/val'
], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f'Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}')

In [ ]:
# Optional: inspect one batch (src, trg, gt_src, gt_trg)
d = next(iter(train_loader))
print([x.shape for x in d])

In [ ]:
from lstm_improved import ImprovedLSTMPredictor
from loss import LossFunction
from torch import optim

DEVICE = 'cuda'
model = ImprovedLSTMPredictor(
    input_dim=13,
    output_dim=5,
    d_model=256,
    hidden_dim=256,
    num_layers=2,
    dropout=0.1,
    teacher_forcing_ratio=0,
).to(DEVICE)
criterion = LossFunction(loss1_coeff=1, loss2_coeff=0, loss3_coeff=0, loss4_coeff=0)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Optional: load a pretrained LSTM checkpoint
# model.load_weight('pretrained/lstm-improved.pth')

In [ ]:
model.evaluate(val_loader, criterion)

In [ ]:
LR = 2e-3
NUM_EPOCHS = 20

optimizer = optim.AdamW(
    model.parameters(),
    lr=LR,
    betas=(0.9, 0.999),
    weight_decay=1e-4,
    eps=1e-8
)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS + 1)
best_val_loss = float("inf")
log_file = open('file_lstm.log', 'w')
log_file.close()

for epoch in range(1, NUM_EPOCHS + 1):
    train_loss = model.train_one_epoch(train_loader, optimizer, criterion, device=DEVICE)
    val_loss = model.evaluate(val_loader, criterion, device=DEVICE)

    scheduler.step()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        model.save_weight('pretrained/lstm-new-d256-h256-e3-d3.pth')

    current_lr = scheduler.get_last_lr()[0]
    print(f"Epoch {epoch}: Train Loss = {train_loss:.8f}, Val Loss = {val_loss:.8f}, LR = {current_lr:.8f}")
    log_file = open('file_lstm.log', 'a')
    log_file.write(f"Epoch {epoch}: Train Loss = {train_loss:.8f}, Val Loss = {val_loss:.8f}, LR = {current_lr:.8f}\n")
    log_file.close()

print("Training complete. Best Val Loss:", best_val_loss)